In [2]:
import sys
sys.path.insert(0, "..")

import numpy as np
from matplotlib import pyplot as plt

from module.base.network import Network
from module.simulation.meanfield import MeanField
import module.components.CONST as CONST
from module.components.lawrence_dist import LawrenceDist

In [3]:
A = np.array([[1,2,3],[4,5,6]])

In [4]:
A.flatten()

array([1, 2, 3, 4, 5, 6])

In [5]:
A.reshape((-1,))

array([1, 2, 3, 4, 5, 6])